# Mexican federal budget pre-processing pipeline

## Instructions

To you run the notebook:

1. choose a unique `ITERATION_LABEL` for each pipeline run
2. specify and describe your input files (`INPUT_FILES`)
3. make sure your column mapping (`COLUMN_ALIASES`) is correct
3. run the whole notebook by clicking on __Kernel > Restart & Run All__

## Settings

Choose a unique iteration label for each pipeline run.

In [1]:
ITERATION_LABEL = 'iteration-6-development'

Put your input files inside the `pipeline.in` folder and describe them here.

In [2]:
INPUT_FILES = {
    2010: {'name': 'Cuenta_Publica_2010.csv', 'encoding': 'windows-1252'},
#     2011: {'name': 'Cuenta_Publica_2011.csv', 'encoding': 'windows-1252'},
#     2012: {'name': 'Cuenta_Publica_2012.csv', 'encoding': 'windows-1252'},
#     2013: {'name': 'Cuenta_Publica_2013.csv', 'encoding': 'windows-1252'},
#     2014: {'name': 'Cuenta_Publica_2014.csv', 'encoding': 'windows-1252'},
#     2015: {'name': 'Cuenta_Publica_2015.csv', 'encoding': 'windows-1252'},
#     2016: {'name': 'PEF2016_AC01.csv', 'encoding': 'cp850'}
}

If your input files don't all have the same column names, define your mapping here. 

In [3]:
COLUMN_ALIASES = {
    'Actividad Institucional': ['AI'],
    'Adefas': ['ADEFAS'],
    'Aprobado': [
        'PEF_2016',
        'Importe Presupuesto de Egresos de la Federación',
        'Importe Presupuesto de Egresos de la Federación (PEF)'
    ],
    'Ciclo': None,
    'Clave de cartera': ['CLAVE_CARTERA'],
    'Descripción de Fuente de Financiamiento': ['FUENTE_FINAN_DESCRIPCION'],
    'Descripción de Función': ['FUNCIONL_DESCRIPCION'],
    'Descripción de Grupo Funcional': [
        'Descripción de Finalidad',
        'GRUPO_FUN_DESCRIPCION',
        'Descripción de Grupo Funcional'
    ],
    'Descripción de Objeto del Gasto': ['CONCEPTO_DESCRIPCION'],
    'Descripción de Programa Presupuestario': ['PROGR_PRES_DESCRIPCION'],
    'Descripción de Ramo': ['RAMO_DESCRIPCION'],
    'Descripción de Reasignacion': ['REASIGNACION_DESCRIPCION'],
    'Descripción de Subfunción': ['SUBFUNCIONL_DESCRIPCION'],
    'Descripción de Tipo de Gasto': ['TIPO_GASTO_DESCRIPCION'],
    'Descripción de Unidad Responsable': ['UNIDAD_DESCRIPCION'],
    'Descripción de la Actividad Institucional': [
        'ACTIVIDAD_INST_DESCRIPCION',
        'Descripción de Actividad Institucional'
    ],
    'Descripción de la entidad federativa': ['ENTIDAD_FED_DESCRIPCION'],
    'Descripción de la modalidad del programa presupuestario': [
        'MODALIDAD_DESCRIPCION',
        'Descripción del Identificador del Programa Presupuestario',
        'Descripción del Identificador de Programa Presupuestario'
    ],
    'Devengado': None,
    'Ejercicio': None,
    'Ejercido': None,
    'Entidad Federativa': ['EF'],
    'Fuente de Financiamiento': ['FF'],
    'Función': ['FN'],
    'Grupo Funcional': [
        'Finalidad', 'GF', 'Grupo Funcional'
    ],
    'Modalidad del Programa presupuestario': [
        'MOD',
        'Identificador de Programa Presupuestario',
        'Identificador del Programa Presupuestario'
    ],
    'Modificado': None,
    'Objeto del Gasto': ['CONCEPTO'],
    'Pagado': None,
    'Programa Presupuestario': ['PP'],
    'Ramo': None,
    'Reasignacion': ['RA'],
    'Subfunción': ['SF'],
    'Tipo de Gasto': ['TG'],
    'Unidad Responsable': ['UNIDAD']
}

The following hierarchical categories will have IDs prefixed with the parent categories:

In [4]:
HIERARCHIES = {
    'functional': [
        'Grupo Funcional', 
        'Función', 
        'Subfunción', 
        'Actividad Institucional'
    ],
#     'administrative': [
#         'Ramo', 
#         'Unidad Responsable'
#     ],
#     'activities': [
#         'Modalidad del Programa presupuestario', 
#         'Programa Presupuestario'
#     ],
}

That's it. Now just run the notebook from beginning to end.

## Imports

In [5]:
from sys import stdout
from pandas import read_csv, concat, DataFrame, ExcelWriter, ExcelFile, Series
from numpy import nan
from os.path import join, isdir
from os import mkdir
from json import dumps
from pprint import pprint
from slugify import slugify

## Configuration

In [6]:
BASENAME = 'mexican_federal_budget'
INPUT_FOLDER = 'pipeline.in'
OUTPUT_FOLDER = 'pipeline.out'
ITERATION_FOLDER = join(OUTPUT_FOLDER, ITERATION_LABEL)
MERGED_FILE = join(ITERATION_FOLDER, BASENAME + '.merged.csv')
CATALOGS_FOLDER = 'objeto_del_gasto.catalog'
CATALOGS_FILE = 'objeto_del_gasto.catalog.xlsx'

In [7]:
if isdir(ITERATION_FOLDER):
    raise ValueError('Please enter a unique iteration label')
    
mkdir(ITERATION_FOLDER)

## Encoding inspection

Detect the file encodings of the input files using the `cChardet` utility library. __Warning:__ it's not always accurate. This is meant only as an indication only. In the end, encodings will be taken from `INPUT_FILES`.

In [8]:
def detect_encodings():
    """Detect CSV file encoding with the cChardet library"""

    try:
        import cchardet as chardet
    except ImportError:
        cChardet = 'https://github.com/PyYoshi/cChardet'
        print('Encoding inspection skipped: install %s', cChardet)
        return

    results = {}
    results_file = join(OUTPUT_FOLDER, ITERATION_LABEL, 'encodings.detected.json')
    
    for year, file in sorted(INPUT_FILES.items()):
        datafile = join(INPUT_FOLDER, file['name'])
        
        with open(datafile, 'rb') as f:
            text = f.read()
            
        result = chardet.detect(text)
        results.update({year: result})
        print(year, 'Inspected', file['name'], result)
    
    with open(results_file, 'w+') as json:
        json.write(dumps(results, indent=4))
        print('\nSaved encoding detection report to', results_file)
        
# detect_encodings()

## Load files

In [9]:
def read_columns(file, encoding):
    """Return clean CSV file headers"""
    
    with open(file, encoding=encoding) as csv:
        header = csv.readline()
        return header.replace('\n', '').split(',')

In [10]:
def force_strings(columns):
    """Return string enforcement for each column of a CSV file"""
    
    for column in columns:
        yield column, str

In [11]:
def load_csv_files():
    """Load raw data (CSV) files"""
    
    batch = {}
    
    for year, file in sorted(INPUT_FILES.items()):
        filepath = join(INPUT_FOLDER, file['name'])
        column_names = read_columns(filepath, file['encoding'])
        column_types = dict(force_strings(column_names))
        
        batch[year] = read_csv(filepath, encoding=file['encoding'], dtype=column_types)
        print('Loaded', file['name'], 'with encoding', file['encoding'])
        stdout.flush()
            
    return batch

## Clean the data

In [12]:
def strip_cell_padding(batch):
    for year in sorted(batch.keys()):
        for column in batch[year].columns:
            batch[year].rename(columns={column: column.strip()}, inplace=True)
            batch[year][column] = batch[year][column].apply(lambda x: x.strip() if x is not nan else x)
        print(year, 'stripped cell paddings')
        stdout.flush()

In [13]:
def delete_empty_columns(batch):
    for year in batch.keys():
        for column in batch[year].columns:
            if 'Unnamed:' in column:
                try:
                    del batch[year][column]
                    print(year, column, 'deleted')
                    stdout.flush()
                except KeyError:
                    pass  

In [14]:
def count_missing_values(batch):
    collector = {}
    table = []

    for column in get_union_of_columns(batch):
        row = {'Column': column}
        collector.update({column: []})
        
        for year in batch.keys():
            if column in batch[year].columns:
                is_empty = batch[year][column].isnull()
                empty_lines = batch[year].where(is_empty).dropna(how='all')
                collector[column].extend(empty_lines.to_dict(orient='records'))
                nb_empty_cells = len(empty_lines)
            else:
                nb_empty_cells = nan
                
            row.update({year: nb_empty_cells})
            if nb_empty_cells not in (nan, 0):
                print(year, 'found', nb_empty_cells, 'missing values in', column)

        table.append(row)
        
    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    empty_values_overview_table = DataFrame(table).reindex_axis(ordered_columns, axis=1)
    
    return empty_values_overview_table, collector

In [15]:
def count_duplicates(batch):
    for year, df in sorted(batch.items()):
        nb_duplicate_lines = df.duplicated().apply(lambda x: 1 if x is True else 0).sum()
        print(year, 'found', nb_duplicate_lines, 'duplicate lines')

## Alias column names

In [16]:
def get_union_of_columns(batch):
    union = set()
    for year in batch.keys():
        union = union | set(batch[year].columns)
    return union

In [17]:
from yaml import load

def load_aliases(file):
    with open(file) as yaml:
        aliases = load(yaml.read())
        return aliases

In [18]:
def map_columns_to_aliases(batch, list_of_aliases):
    for year in sorted(batch.keys()):
        for column in sorted(batch[year].columns):
            if not column in list_of_aliases:
                for reference, aliases in list_of_aliases.items():
                    if aliases:
                        if column in aliases:
                            batch[year].rename(columns={column: reference}, inplace=True)
                            print(year, column, 'replaced with', reference)
                            stdout.flush()
                            break  
                else:
                    print(year, 'NO ALIAS: ', column)
                    stdout.flush()

In [19]:
def build_overview(batch):
    table = []
    
    for column in get_union_of_columns(batch):
        row = {'Column': column}
        for year in batch.keys():
            row.update({year: column in batch[year].columns})
        table.append(row)
        
    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    
    overview = DataFrame(table).reindex_axis(ordered_columns, axis=1)
    return overview

## Check expenditure sums

There's a little cleaning to do on the amount columns (zeros represented by a dash). Assume thousands are seperated by a comma.

In [20]:
EXPENDITURE_COLUMNS = [
    'Ejercido', 
    'Devengado', 
    'Aprobado', 
    'Pagado', 
    'Modificado', 
    'Adefas', 
    'Ejercicio'
]

def clean_expenditure_columns(batch):
    check_sums = []

    for column in EXPENDITURE_COLUMNS:
        row = {'Column': column}
        
        for year in sorted(batch.keys()):
            try:
                series = batch[year][column]
                
                # I'm assuming -' represents zero
                series = series.apply(lambda x: '0' if x == '-' else x)
                series = series.apply(lambda x: x.replace(',', '') if x is not nan else x)                
                batch[year][column] = series.astype(float)
                check_sum = batch[year][column].sum()
                
                print(year, 'cleaned and summed', column, '=', check_sum, 'pesos')
                
            except KeyError:
                check_sum = nan
                
            row.update({year: check_sum})
        
        check_sums.append(row)

    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    return DataFrame(check_sums).reindex_axis(ordered_columns, axis=1)    

## Objeto del Gasto Column split

In [21]:
from os.path import join

def generate_catalog(file):
    
    catalog_ = {}
    catalog_file = ExcelFile(file)
    INDEX_COLUMN = 0
    
    for sheet in catalog_file.sheet_names:
        if sheet != 'Concatenated':
            name = slugify(sheet, separator='_')
            output = join('objeto_del_gasto.catalog', name + '.csv')

            df = catalog_file.parse(sheet).dropna()
            index = df.columns[INDEX_COLUMN]

            df[index] =  df[index].astype(str)
            df.set_index(index, inplace=True)
            df = df.groupby(df.index).first()
            df.sort_index(inplace=True)
            
            message = 'Loaded catalog {sheet} into "{name}" ({nb} lines)'
            parameters = dict(sheet=sheet, name=name, nb=len(df))

            print(message.format(**parameters))
            catalog_[name] = df['DESCRIPCION']
    
    print()
    return catalog_

In [22]:
c = generate_catalog(CATALOGS_FILE)
c['concepto']

Loaded catalog CAPITULO into "capitulo" (9 lines)
Loaded catalog CONCEPTO into "concepto" (63 lines)
Loaded catalog PARTIDA GENERICA into "partida_generica" (354 lines)
Loaded catalog PARTIDA ESPECÍFICA into "partida_especifica" (668 lines)



CONCEPTO
1100    Remuneraciones al personal de carácter permanente
1200    Remuneraciones al personal de carácter transit...
1300              Remuneraciones adicionales y especiales
1400                                     Seguridad social
1500             Otras prestaciones sociales y económicas
1600                                          Previsiones
1700              Pago de estímulos a servidores públicos
2100    Materiales de administracion, emision de docum...
2200                               Alimentos y utensilios
2300    Materias primas y materiales de produccion y c...
2400    Materiales y articulos de construccion y de re...
2500    Productos químicos, farmacéuticos y de laborat...
2600                 Combustibles, lubricantes y aditivos
2700    Vestuario, blancos, prendas de protección y ar...
2800              Materiales y suministros para seguridad
2900       Herramientas, refacciones y accesorios menores
3100                                    Servicios basicos
3200 

In [23]:
def split_objeto_del_gasto(batch):
    catalog = generate_catalog(CATALOGS_FILE)
    missing_in_catalog = []

    def has_digits(n, N):
        return n is not nan and len(n) >= N 

    def lookup(n, table, year):
        try:
            return catalog[table].loc[n]
        except KeyError:
            missing_in_catalog.append({'year': year, 'table': table, 'ID': n})
            return nan
        except TypeError:
            # n is nan
            return nan


    for year in sorted(batch.keys()):
        print(year, 'splitting objeto del gasto column')
        objeto = batch[year]['Objeto del Gasto'].astype(str)
        
        batch[year]['Capitulo'] = objeto.apply(lambda x: x[0] + '000' if x not in (nan, 'nan') else nan)
        batch[year]['Concepto'] = objeto.apply(lambda x: x[:2] + '00' if x not in (nan, 'nan') else nan)
        batch[year]['Partida Genérica'] = objeto.apply(lambda x: x[:3] if has_digits(x, 4) else nan)
        batch[year]['Partida Específica'] = objeto.apply(lambda x: x if has_digits(x, 5) else nan)

        batch[year]['Descripción de Capitulo'] = batch[year]['Capitulo'].map(lambda x: lookup(x, 'capitulo', year))  
        batch[year]['Descripción de Concepto'] = batch[year]['Concepto'].map(lambda x: lookup(x, 'concepto', year))  
        batch[year]['Descripción de Partida Genérica'] = batch[year]['Partida Genérica'].map(lambda x: lookup(x, 'partida_generica', year))  
        batch[year]['Descripción de Partida Específica'] = batch[year]['Partida Específica'].map(lambda x: lookup(x, 'partida_especifica', year))  
        
    return DataFrame(missing_in_catalog).drop_duplicates()

## Prefix IDs 
Disambiguating sub-categories may require prefixing their IDs with their parents' IDs.

In [24]:
def prefix_ids(batch):
    for year in batch.keys():       
        for hierarchy, levels in HIERARCHIES.items():
            prefix = batch[year]['Ciclo'].apply(lambda x: '')
            for n, level in enumerate(levels):
                dash = '-' if n > 0 else ''
                prefix = prefix + dash + batch[year][level]  
                batch[year][level] = prefix
                
                print(year, 'prefixed', hierarchy, 'level', n, level)

##  Pipeline

In [25]:
def do_pipeline():

    def echo_section(section):
        print('\n', section, '\n')

    echo_section('Loading files')
    datasets = load_csv_files()
    
    echo_section('Delete empty columns')
    delete_empty_columns(datasets)

    echo_section('Stripping padding from cells')
    strip_cell_padding(datasets)
    
    echo_section('Counting duplicate lines (NOT de-duplicating)')
    count_duplicates(datasets)
    
    echo_section('Mapping column to aliases')
    map_columns_to_aliases(datasets, COLUMN_ALIASES)

    echo_section('Counting missing values')
    missing_values_report, bad_records = count_missing_values(datasets)
    
    echo_section('Building column mapping overview')
    column_mapping_report = build_overview(datasets)
    
    echo_section('Cleaning expenditure columns')
    sums_report = clean_expenditure_columns(datasets)
    
    echo_section('Breaking down Objeto del Gasto column')
    missing_catalog_ids = split_objeto_del_gasto(datasets)
        
    echo_section('Prefixing IDs of certain category hierarchies')
    prefix_ids(datasets)

    echo_section('Saving pipeline configuration')

    reports_file = join(ITERATION_FOLDER, BASENAME + '.reports.xlsx')
    writer = ExcelWriter(reports_file)    
    missing_values_report.to_excel(writer, 'missing values', encoding='utf-8', index=False)
    column_mapping_report.to_excel(writer, 'column mapping', encoding='utf-8', index=False)
    sums_report.to_excel(writer, 'check sums', encoding='utf-8', index=False)
    missing_catalog_ids.to_excel(writer, 'missing_catalog_IDs', encoding='utf-8', index=False)    
    print('Saved 4 reports to', reports_file)    

    aliases_file = join(ITERATION_FOLDER, BASENAME + '.aliases.json')
    inputs_file = join(ITERATION_FOLDER, BASENAME + '.inputs.json')
    levels_file = join(ITERATION_FOLDER, BASENAME + '.levels.json')
    bad_records_file = join(ITERATION_FOLDER, BASENAME + '.missing.json')

    with open(bad_records_file, 'w+') as json:
        json.write(dumps(bad_records, indent=4))
        
    with open(aliases_file, 'w+') as json:
        json.write(dumps(COLUMN_ALIASES, indent=4))
        
    with open(levels_file, 'w+') as json:
        json.write(dumps(HIERARCHIES, indent=4))
        
    with open(inputs_file, 'w+') as json:
        json.write(dumps(INPUT_FILES, indent=4))
    
    print('Saved input configuration to', inputs_file)    
    print('Saved column aliases to', aliases_file) 
    print('Saved bad records (those with empty cells) to', bad_records_file)    
    print('Saved hierarchy levels used for prefixing to', levels_file) 
    
    echo_section('Pipeline run "%s" done' % ITERATION_LABEL)

    return missing_catalog_ids, column_mapping_report, missing_values_report, sums_report, datasets

## Run the pipeline

In [26]:
missing_ids, column_mapping, missing_values, sums, budgets = do_pipeline()


 Loading files 

Loaded Cuenta_Publica_2010.csv with encoding windows-1252

 Delete empty columns 


 Stripping padding from cells 

2010 stripped cell paddings

 Counting duplicate lines (NOT de-duplicating) 

2010 found 34 duplicate lines

 Mapping column to aliases 

2010 Descripción de Finalidad replaced with Descripción de Grupo Funcional
2010 Finalidad replaced with Grupo Funcional

 Counting missing values 


 Building column mapping overview 


 Cleaning expenditure columns 

2010 cleaned and summed Ejercido = 2.47409995292e+12 pesos
2010 cleaned and summed Aprobado = 2.37691503329e+12 pesos

 Breaking down Objeto del Gasto column 

Loaded catalog CAPITULO into "capitulo" (9 lines)
Loaded catalog CONCEPTO into "concepto" (63 lines)
Loaded catalog PARTIDA GENERICA into "partida_generica" (354 lines)
Loaded catalog PARTIDA ESPECÍFICA into "partida_especifica" (668 lines)

2010 splitting objeto del gasto column

 Prefixing IDs of certain category hierarchies 

2010 prefixed funct

In [27]:
from gc import collect
collect()

4522

In [28]:
for year, budget in budgets.items():
    filepath = MERGED_FILE.replace('merged', str(year))
    budget.to_csv(filepath, encoding='utf-8', index=False)
    print('Saved', filepath)

Saved pipeline.out/iteration-6-development/mexican_federal_budget.2010.csv


In [29]:
merged = concat(list(budgets.values()))
merged.to_csv(MERGED_FILE, encoding='utf-8', index=False)
print('Saved merged dataset to', MERGED_FILE)    

Saved merged dataset to pipeline.out/iteration-6-development/mexican_federal_budget.merged.csv


## Quality control

In [30]:
sorted(list(merged_budget.columns))

NameError: name 'merged_budget' is not defined

In [ ]:
merged_budget.sample(n=10)

In [ ]:
objeto_breakdown = [
    'Ciclo', 
    'Objeto del Gasto', 
    'Capitulo', 'Concepto', 
    'Partida Específica', 
    'Partida Genérica'
]
merged_budget[objeto_breakdown].sample(n=20)

In [ ]:
print('Total: missing', len(missing_ids), 'catalog IDs to breakdown the "Objeto del Gasto" column')
print('Tables:', dict(missing_ids.groupby('table').count()['ID']))
print('Years:', dict(missing_ids.groupby('year').count()['ID']))
missing_ids.sample(n=20)

In [ ]:
list(missing_ids['ID'].values)